## 2 Model Baseline

This notebook will get the baseline WER and CER scores for Romansh. For this the test set of the Sursilvan idiom will be decoded with both OpenAI's Whisper and Meta's Omnilingual. While Whisper is not trained on Romansh, Omnilingual is trained on Sursilvan.

In [ ]:
import os
import torch
import whisper
import pandas as pd
import numpy as np
from omnilingual_asr.models.inference.pipeline import ASRInferencePipeline
from omnilingual_asr.models.wav2vec2_llama.lang_ids import supported_langs
from tqdm import tqdm
from jiwer import wer, cer

DATA_PATH = "romansh-data/sursilvan-mini/"
WHISPER_MODEL = "medium"
OMNILINGUAL_MODEL = "omniASR_CTC_1B_v2"
LANGUAGE_CODE = "roh_Latn_surs1244"
BATCH_SIZE = 8
NUM_TEST_SAMPLES = None

print("="*60)
print("Combined ASR Baseline: Whisper vs Omnilingual")
print("="*60)

Combined ASR Baseline: Whisper vs Omnilingual


In [ ]:
def load_test_data(data_path, num_samples=None):
    """Load test data"""
    test_tsv = os.path.join(data_path, "test.tsv")
    clips_path = os.path.join(data_path, "clips")
    
    print(f"\n📂 Loading test data from: {test_tsv}")
    df = pd.read_csv(test_tsv, sep='\t')
    
    if num_samples:
        df = df.head(num_samples)
        print(f"   Using {num_samples} samples for quick test")

    audio_paths = []
    valid_indices = []
    
    for idx, row in df.iterrows():
        audio_path = os.path.join(clips_path, row['path'])
        if os.path.exists(audio_path):
            audio_paths.append(audio_path)
            valid_indices.append(idx)
    
    df = df.loc[valid_indices].reset_index(drop=True)
    
    print(f"   Total utterances: {len(df)}")
    print(f"   Audio files found: {len(audio_paths)}/{len(df)}")
    
    return df, audio_paths

df_test, audio_paths = load_test_data(DATA_PATH, NUM_TEST_SAMPLES)
references = df_test['sentence'].tolist()


📂 Loading test data from: romansh-data/sursilvan-mini/test.tsv
   Total utterances: 94
   Audio files found: 94/94


In [ ]:
print("\n" + "="*60)
print("🎙️ Whisper Evaluation")
print("="*60)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

print(f"\n📥 Loading Whisper {WHISPER_MODEL}...")
whisper_model = whisper.load_model(WHISPER_MODEL, device=device)
print("✅ Whisper loaded")

print(f"\n🎙️ Transcribing {len(audio_paths)} files with Whisper...")
whisper_transcriptions = []
whisper_languages = []

for audio_path in tqdm(audio_paths, desc="Whisper"):
    try:
        result = whisper_model.transcribe(
            audio_path,
            task="transcribe",
            fp16=torch.cuda.is_available(),
            verbose=False
        )
        whisper_transcriptions.append(result['text'].strip())
        whisper_languages.append(result.get('language', 'unknown'))
    except Exception as e:
        whisper_transcriptions.append("")
        whisper_languages.append("error")
        print(f"⚠️ Error: {e}")


🎙️ Whisper Evaluation
Device: cuda

📥 Loading Whisper medium...
✅ Whisper loaded

🎙️ Transcribing 94 files with Whisper...


Whisper:   0%|          | 0/94 [00:00<?, ?it/s]

Detected language: German


Whisper:   1%|          | 1/94 [00:05<08:25,  5.43s/it]

Detected language: Slovenian


...

Whisper:  98%|█████████▊| 92/94 [10:09<00:12,  6.26s/it]

Detected language: Yiddish


Whisper:  99%|█████████▉| 93/94 [10:12<00:05,  5.22s/it]

Detected language: Yiddish


Whisper: 100%|██████████| 94/94 [10:17<00:00,  6.57s/it]


In [ ]:
print("\n" + "="*60)
print("🌐 Omnilingual Evaluation")
print("="*60)

print(f"Language code: {LANGUAGE_CODE}")
if LANGUAGE_CODE in supported_langs:
    print(f"✅ Romansh supported")
else:
    print(f"⚠️ Romansh not in supported list")

print(f"\n📥 Loading Omnilingual {OMNILINGUAL_MODEL}...")
omnilingual_pipeline = ASRInferencePipeline(model_card=OMNILINGUAL_MODEL)
print("✅ Omnilingual loaded")

print(f"\n🎙️ Transcribing {len(audio_paths)} files with Omnilingual...")
omnilingual_transcriptions = []

for i in range(0, len(audio_paths), BATCH_SIZE):
    batch_paths = audio_paths[i:i+BATCH_SIZE]
    try:
        results = omnilingual_pipeline.transcribe(
            batch_paths,
            lang=[LANGUAGE_CODE] * len(batch_paths),
            batch_size=len(batch_paths)
        )
        omnilingual_transcriptions.extend(results)
    except Exception as e:
        print(f"⚠️ Batch error: {e}")
        omnilingual_transcriptions.extend([""] * len(batch_paths))
    
    print(f"   Processed {min(i+BATCH_SIZE, len(audio_paths))}/{len(audio_paths)} files")


🌐 Omnilingual Evaluation
Language code: roh_Latn_surs1244
✅ Romansh supported

📥 Loading Omnilingual omniASR_CTC_1B_v2...


Output()

✅ Omnilingual loaded

🎙️ Transcribing 94 files with Omnilingual...
   Processed 8/94 files
   Processed 16/94 files
   Processed 24/94 files
   Processed 32/94 files
   Processed 40/94 files
   Processed 48/94 files
   Processed 56/94 files
   Processed 64/94 files
   Processed 72/94 files
   Processed 80/94 files
   Processed 88/94 files
   Processed 94/94 files


In [ ]:
def calculate_metrics(references, hypotheses, model_name):
    """Calculate WER and CER"""
    valid_pairs = [(ref, hyp) for ref, hyp in zip(references, hypotheses) 
                   if ref and hyp]
    
    if not valid_pairs:
        return None, None, 0
    
    refs = [pair[0] for pair in valid_pairs]
    hyps = [pair[1] for pair in valid_pairs]
    
    wer_score = wer(refs, hyps)
    cer_score = cer(refs, hyps)
    
    return wer_score, cer_score, len(valid_pairs)

whisper_wer, whisper_cer, whisper_valid = calculate_metrics(
    references, whisper_transcriptions, "Whisper"
)

omnilingual_wer, omnilingual_cer, omnilingual_valid = calculate_metrics(
    references, omnilingual_transcriptions, "Omnilingual"
)

In [ ]:
print("\n" + "="*60)
print("📊 FINAL RESULTS COMPARISON")
print("="*60)

print(f"\nTest set size: {len(df_test)} utterances")
print(f"Valid for Whisper: {whisper_valid}/{len(df_test)}")
print(f"Valid for Omnilingual: {omnilingual_valid}/{len(df_test)}")

print("\n" + "-"*40)
print("WHISPER (auto-detection)")
print("-"*40)
if whisper_wer is not None:
    print(f"WER: {whisper_wer:.4f} ({whisper_wer*100:.2f}%)")
    print(f"CER: {whisper_cer:.4f} ({whisper_cer*100:.2f}%)")
else:
    print("❌ No valid transcriptions")

print("\n" + "-"*40)
print("OMNILINGUAL (native Romansh)")
print("-"*40)
if omnilingual_wer is not None:
    print(f"WER: {omnilingual_wer:.4f} ({omnilingual_wer*100:.2f}%)")
    print(f"CER: {omnilingual_cer:.4f} ({omnilingual_cer*100:.2f}%)")
else:
    print("❌ No valid transcriptions")

if whisper_wer is not None and omnilingual_wer is not None:
    improvement = ((whisper_wer - omnilingual_wer) / whisper_wer) * 100
    print("\n" + "-"*40)
    print("COMPARISON")
    print("-"*40)
    if omnilingual_wer < whisper_wer:
        print(f"✅ Omnilingual is {improvement:.1f}% better than Whisper")
    elif omnilingual_wer > whisper_wer:
        print(f"❌ Whisper is {abs(improvement):.1f}% better than Omnilingual")
    else:
        print("📊 Both models perform equally")


📊 FINAL RESULTS COMPARISON

Test set size: 94 utterances
Valid for Whisper: 90/94
Valid for Omnilingual: 94/94

----------------------------------------
WHISPER (auto-detection)
----------------------------------------
WER: 0.9851 (98.51%)
CER: 0.5879 (58.79%)

----------------------------------------
OMNILINGUAL (native Romansh)
----------------------------------------
WER: 0.7585 (75.85%)
CER: 0.2422 (24.22%)

----------------------------------------
COMPARISON
----------------------------------------
✅ Omnilingual is 23.0% better than Whisper


In [ ]:
print("\n" + "="*60)
print("📝 EXAMPLE TRANSCRIPTIONS (First 3 files)")
print("="*60)

for i in range(min(3, len(df_test))):
    print(f"\n--- Example {i+1} ---")
    print(f"Reference: {references[i][:200]}...")
    print(f"\nWhisper ({whisper_languages[i]}):")
    print(f"  {whisper_transcriptions[i][:200]}...")
    print(f"\nOmnilingual:")
    print(f"  {omnilingual_transcriptions[i][:200]}...")
    
    if references[i] and whisper_transcriptions[i] and omnilingual_transcriptions[i]:
        try:
            w_wer = wer(references[i], whisper_transcriptions[i])
            o_wer = wer(references[i], omnilingual_transcriptions[i])
            print(f"\n  WER: Whisper={w_wer:.3f}, Omnilingual={o_wer:.3f}")
        except:
            pass
    print("-"*40)


📝 EXAMPLE TRANSCRIPTIONS (First 3 files)

--- Example 1 ---
Reference: lgl agid svizzer per la muntogna sustegn ils purs grischuns. A Tartar sin la Muntogna han responsabels digl agid da muntogna surdau oz in schec da 3 100 1000 francs pils purs ch'han stuiu cumprar pave...

Whisper (de):
  Die Jägide Schweizer Perla Montagne susteign il Spurs Grisuns. Tartar sind le Montagne an responsables de jägide Montagne sur d'Ao zinschekte 3.000 Fr. spils Spurs kann stu kumpra Pavel murt l'aschiti...

Omnilingual:
  l liegit svizer per la muntogna sustegn ils purs grijchuns a tartar sin la muntogna han responsabels di gliegit da muntogna sur dauots in scec da treic meli francs pils purs ch'an stü cumprà pavel muo...

  WER: Whisper=0.934, Omnilingual=0.618
----------------------------------------

--- Example 2 ---
Reference: II possessur dalla Casa Demont a Vella sto pagar il castitg maximal per haver spazzau illegalmein quella casa. 30 1000 francs cuosta ei ad el d'haver destruiu la casa da v

In [ ]:
print("\n" + "="*60)
print("🌍 Whisper Language Detection")
print("="*60)

if whisper_languages:
    from collections import Counter
    lang_counts = Counter(whisper_languages)
    total = len(whisper_languages)
    
    for lang, count in lang_counts.most_common():
        print(f"  {lang}: {count} ({count/total*100:.1f}%)")


🌍 Whisper Language Detection
  de: 33 (35.1%)
  sl: 22 (23.4%)
  oc: 9 (9.6%)
  it: 6 (6.4%)
  la: 5 (5.3%)
  sq: 3 (3.2%)
  nl: 3 (3.2%)
  cy: 3 (3.2%)
  yi: 3 (3.2%)
  br: 1 (1.1%)
  hu: 1 (1.1%)
  en: 1 (1.1%)
  pt: 1 (1.1%)
  he: 1 (1.1%)
  lt: 1 (1.1%)
  fr: 1 (1.1%)


: 